In [7]:
import pandas as pd
import datetime

def spreadsheet_filter(spreadsheet, lang, date_1,date_2):
    
    '''This takes the DRM SIO data, and makes it useful for sorting out files into folders'''
    
    audio_data_set = pd.read_excel(spreadsheet)
    
    #audio_data_set.drop(0, inplace=True)
    
    audio_data_set.rename(columns= {'FH':'Location', 'Net':'Language' , 'ObDate':'Date' , 'ObTM':'Time' , 'STA':'S' , 'S':'I' , 
                                    'O':'p1','D':'O',  'Freq': 'Region', 'Lang':'Freq' }, inplace= True)
    print(audio_data_set)
    
    # this filter the language specified(code the columns properly later)
    lang_flt = audio_data_set.loc[audio_data_set["Language"] == lang] 
    
    #takes the date range and filters the data.Produces the sorted data into excel file
    
    date_flt = lang_flt[lang_flt["Date"].dt.strftime("%Y-%m-%d").between(date_1,date_2)]
    date_flt.to_excel('Sorted_data_{}_{}_{}_.xlsx'.format(lang,date_1,date_2))
    
    date_region_SIO = date_flt[["Date", "Time", "Location", "S", "I", "O" ]]
    
    d_r_SIO = date_flt[["Date", "Location", "Freq", "S", "I", "O" ]]  #streamlining data set, w/o time
    time_4_dig = []

    for i in date_region_SIO.index:  # altering the format of the times that have 3 digits e.g 501 should be 0501 
        #(meaning 05:01)
        if len(str(date_region_SIO["Time"][i])) == 3:
            time_4_dig.append("0"+str(date_region_SIO["Time"][i]))
        else:
            time_4_dig.append(str(date_region_SIO["Time"][i]))
    
    d_r_SIO.insert(1, "Time", time_4_dig, True)  # adding the correct time to the spreadhseet     

    
    date_combine = date_flt["Date"].dt.strftime('%Y%m%d')  # altering the data to be YYYYMMDDD not YYYY-MM-DD

    location_rename = {"ACC":"ACCR", "ADD": "ADDI", "HAR" : "HARA", "MAI":"MAID", 
                       "NAI":"NAIR",'JOB':'JOBG', 'MDG':'MADG', "DUS":"DUSH", "ISL": "ISLA", "KAB":"KABU"
                      , "MCL":"MCLO", "QUE":"QUET","KHI":"KARA", "TUN":"TUNI", "MAI":"MAID", "BAN":"BANJ",
                      }  # will need to generate a dictionary function for this

    d_r_t_combine = []

    freq_fix = []

    for j in d_r_SIO["Freq"]:
        freq_len = len(str(j))

        if freq_len == 4:
            freq_fix.append('_'+str(j))
        else:
            freq_fix.append(str(j))
            
    d_r_SIO.insert(3,"_Freq", freq_fix)        
    
    
    for i in d_r_SIO.index:

        d_r_t_combine.append(date_combine[i][2:] + d_r_SIO["Time"][i]+ '@'+ 
                             location_rename[d_r_SIO["Location"][i]] +str(d_r_SIO["_Freq"][i]))

    d_r_t_SIO_final = date_flt[[ "S", "I", "O" ]]
    d_r_t_SIO_final.insert(0, "Filename", d_r_t_combine)

    
    d_r_t_SIO_final.to_excel('Filename_and_score_{}_{}_{}.xlsx'.format(lang,date_1,date_2),index=False)
    # this finally makes the spreadsheet with filename and SIO scoring for our sorting
    
    return d_r_t_SIO_final  # return the function to be used in the move_data_dirs function

spreadsheet_filter("Test_set.xlsx", "FREN", "2024-02-12", "2024-02-17")

      Location  WK      FileTime       Date  Time   BC Language  Freq Region  \
0          ACC   1  231101150015 2023-10-31   501  BBC     AFWC  7285    ASC   
1          ACC   1  231101150015 2023-10-31   501  BBC     AFWW  9410    ASC   
2          ACC   1  231101150015 2023-10-31   502  BBC     AFWW  6195    SMG   
3          ACC   1  231101150015 2023-10-31   531  BBC     AFWC  7285    ASC   
4          ACC   1  231101150015 2023-10-31   531  BBC     AFWW  9410    ASC   
...        ...  ..           ...        ...   ...  ...      ...   ...    ...   
51014      YER  16  240217055334 2024-02-16  1632  BBC      WSE  1413    SLA   
51015      YER  16  240217055334 2024-02-16  1901  BBC      WSE  1413    SLA   
51016      YER  16  240217055334 2024-02-16  1931  BBC      WSE  1413    SLA   
51017      YER  16  240217055334 2024-02-16  2001  BBC      WSE  1413    SLA   
51018      YER  16  240217055334 2024-02-16  2031  BBC      WSE  1413    SLA   

       S  I  O p1  F V1   V2    H    C 

,Filename,S,I,O
7483,2402140601@BANJ_7265,4,4,4
7484,2402140601@BANJ_6195,4,4,4
7495,2402141201@BANJ21500,4,4,4
7503,2402141802@BANJ_6195,2,2,2
7518,2402150601@BANJ_7265,4,4,4
7519,2402150601@BANJ_6195,4,4,4
7530,2402151201@BANJ21500,4,4,4
7538,2402151802@BANJ_6195,1,1,1
7551,2402160601@BANJ_7265,4,4,4
7552,2402160601@BANJ_6195,4,4,4


In [2]:
import os 
import shutil
import subprocess

def wav_conv_folder(source_fldr_name):  # let the parent path be everything but the folder. Source folder is just the folder name
    '''The aim is to find mp4 files and move them to mp4 folder. then move convert the files to wav, and then move the files
    to wav_source_files '''
    
    # first i will need to make a list of files with the parent path included
    
    parent_path = "C:/Users/akingv01/OneDrive - BBC/ML_AM_Monitoring"
    source_filepath = os.path.join(parent_path, source_fldr_name)
    #wav_folder_filepath = os.path.join(parent_path,str("WAV_file_source "+source_fldr_name))
    wav_folder_filepath2 = os.path.join(parent_path,str("WAV_file_source (back) "+source_fldr_name))
    
    #os.mkdir(wav_folder_filepath) # this line works so the folder has been deleted
    os.mkdir(wav_folder_filepath2)
    dir_list = os.listdir(source_filepath)
    
    
    #next, I will need to take all files with MP4 and create a new folder called MP4_"insert folder_name"
    # think about stepping to directory down search for the folder move, that would be ideal, or just state the dir for ML??
    
    for i in dir_list:
        if ".MP4" in i:
            
            moving_file_mp4 = os.path.join(source_filepath , i)
            output_wav = moving_file_mp4.replace("MP4", "wav")
            command = ["ffmpeg", "-i", moving_file_mp4, "-vn", "-acodec", "pcm_s16le", "-ar", "44100", "-ac", "1", output_wav]
            subprocess.call(command)
            
            #dst = shutil.move(os.path.join(source_filepath, output_wav) , wav_folder_filepath)
            dst2 = shutil.move(os.path.join(source_filepath, output_wav) , wav_folder_filepath2)
    
    #copy_fldr = os.path.join(parent_path, str("wav_copy"+source_fldr_name))
    #os.mkdir(copy_fldr)
    #shutil.copy(wav_folder_filepath2, copy_fldr)        
    
    
    
    return wav_folder_filepath2, source_fldr_name
            
    
    
wav_conv_folder("French_files")


('C:/Users/akingv01/OneDrive - BBC/ML_AM_Monitoring\\WAV_file_source (back) French_files',
 'French_files')

In [20]:
import os
import shutil
import subprocess

def move_data_dirs(file_folder, drt_SIO):
    
    # make the folder for the sorted files to go into 
    parent_dir = "C:/Users/akingv01/OneDrive - BBC/ML_AM_Monitoring/AM_Monitoring_Classes"
    sorted_scores = str(file_folder[1]) + "Sorted O scores"
    path = os.path.join(parent_dir, sorted_scores)
    os.mkdir(path)
    
    o_score_dir = []
    
    # this makes the folders 1,2,3,4,5
    for i in range(1,6):
        o_score_dir.append(str(i))
    for j in o_score_dir:
        o_score_path = os.path.join(path,j)
        os.mkdir(o_score_path)
        
    source_files = file_folder[0]
    
    filename_list = os.listdir(source_files)
    
    #
    for x in filename_list:
        for y in drt_SIO.index:
            if drt_SIO['Filename'][y] in x:
                moving_file_mp4 = os.path.join(source_files,x)
                output_wav = moving_file_mp4.replace("MP4", "wav")
                command = ["ffmpeg", "-i", moving_file_mp4, "-vn", "-acodec", "pcm_s16le", "-ar", "44100", "-ac", "1", output_wav]
                subprocess.call(command)
                
                
                dst = shutil.move(os.path.join(source_files,x), os.path.join(path,str(drt_SIO["O"][y])))
                print(dst)
    
    
    
    

In [21]:
move_data_dirs(wav_conv_folder("French_files"), spreadsheet_filter("Test_set.xlsx", "FREN", "2024-02-12", "2024-02-16"))

#with this data set there is no 5's but that does not matter a whole lot since there are good files to score. 

      Location  WK      FileTime       Date  Time   BC Language  Freq Region  \
0          ACC   1  231101150015 2023-10-31   501  BBC     AFWC  7285    ASC   
1          ACC   1  231101150015 2023-10-31   501  BBC     AFWW  9410    ASC   
2          ACC   1  231101150015 2023-10-31   502  BBC     AFWW  6195    SMG   
3          ACC   1  231101150015 2023-10-31   531  BBC     AFWC  7285    ASC   
4          ACC   1  231101150015 2023-10-31   531  BBC     AFWW  9410    ASC   
...        ...  ..           ...        ...   ...  ...      ...   ...    ...   
51014      YER  16  240217055334 2024-02-16  1632  BBC      WSE  1413    SLA   
51015      YER  16  240217055334 2024-02-16  1901  BBC      WSE  1413    SLA   
51016      YER  16  240217055334 2024-02-16  1931  BBC      WSE  1413    SLA   
51017      YER  16  240217055334 2024-02-16  2001  BBC      WSE  1413    SLA   
51018      YER  16  240217055334 2024-02-16  2031  BBC      WSE  1413    SLA   

       S  I  O p1  F V1   V2    H    C 

In [22]:
import os
import shutil
import subprocess


def move_add_data(file_folder, drt_SIO):
    par_dir = "C:/Users/akingv01/OneDrive - BBC/ML_AM_Monitoring/AM_Monitoring_Classes"
    
    sort_folder = "Sorted O score folders"
    
    path = os.path.join(par_dir,sort_folder)
    
    source_files = file_folder
    
    filename_list = os.listdir(source_files)
    
    
    for x in filename_list:
        for y in drt_SIO.index:
            if drt_SIO['Filename'][y] in x:
                dst = shutil.move(os.path.join(source_files,x), os.path.join(path,str(drt_SIO["O"][y])))
                print(dst)
